# Quantification of basal body row spacing using Fourier and autocorrelation analysis
This notebook quantifies the average spacing between parallel basal body (BB) rows in Paramecium cells by analyzing maximum projection images of the striated fiber channel from fluorescent microscopy images of regions of interest of the cells. For each image, the code computes the 2D Fourier transform and autocorrelation to identify the dominant periodicity corresponding to the row spacing. The main peak in the mean power spectrum (FFT) and the first peak in the autocorrelation profile (Y direction) are detected to estimate the inter-row distance. Final spacing values are reported based on the FFT analysis, with autocorrelation results provided as additional validation. All results, including plots and summary tables, are saved in an output subfolder for batch processing.

Input:
- image_folder: Folder containing images (e.g., PNG, TIFF) showing parallel striated fiber rows.
- pixel_size_um: Pixel size in microns (for calibration)
- Possible parameters to adjust FFT peak detection sensitivity: prominence, distance in find_peaks function.
- Possible parameters to adjust autocorrelation peak detection sensitivity: height, distance in find_peaks function.

Output:
- For each image: Plots showing the FFT spectrum, mean power spectrum, and autocorrelation profile with detected peaks.
- A CSV file (spacing_results.csv) summarizing the measured spacings and errors for all images.


In [3]:
# BATCH PROCESSING
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft2, fftshift, ifft2, fft, fftfreq
from scipy.signal import find_peaks, peak_widths

# === SETTINGS ===
image_folder = "W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_SR\\Maximum projections\\ROIs for analysis"
pixel_size_um = 0.288  # microns per pixel

# === OUTPUT SETUP ===
output_folder = os.path.join(image_folder, "output")
os.makedirs(output_folder, exist_ok=True)

results = []  # List to store measurement results

# === PROCESS EACH IMAGE IN FOLDER ===
# WORKS BEST WITH PNG IMAGES!
for filename in os.listdir(image_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        continue  # Skip non-image files

    image_path = os.path.join(image_folder, filename)
    print(f"Processing: {filename}")

    # --- Load image in grayscale ---
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not load {filename}. Skipping.")
        continue

    h, w = img.shape

    # === FFT ANALYSIS ===
    f = fft2(img)
    fshift = fftshift(f)
    power_spectrum = np.abs(fshift)**2

    # 1. FFT along vertical axis (Y-direction)
    fft_y = fft(img, axis=0)
    power_y = np.abs(fft_y)**2 # square of the magnitude of the FFT gives the power spectrum --> helps to identify dominant frequencies

    # 2. Mean across image columns
    mean_power_y = fftshift(power_y.mean(axis=1))
    freqs_y = fftshift(fftfreq(h, d=pixel_size_um))  # Frequencies in 1/um

    # 3. Use only positive frequencies
    pos_mask = freqs_y > 0
    pos_freqs = freqs_y[pos_mask]
    pos_power = mean_power_y[pos_mask]

    # 4. Find frequency peaks
    peaks, _ = find_peaks(pos_power, distance=10, prominence=10^7)  # Adjust prominence and distance as needed
    if len(peaks) > 0:

        # # METHOD 1: Find the first peak
        # first_peak_freq = pos_freqs[peaks[0]]
        # spacing_um_fft = 1 / first_peak_freq  # Convert freq to spacing

        # METHOD 2: Find the index of the peak with the maximum power aka the most dominant frequency
        # This gives the most significant frequency component in the image, which corresponds to the dominant spacing
        dominant_peak_idx = peaks[np.argmax(pos_power[peaks])]
        dominant_peak_freq = pos_freqs[dominant_peak_idx]
        spacing_um_fft = 1 / dominant_peak_freq

        # # METHOD 3: peak must be within a certain range
        # # Define frequency range (e.g. expecting spacing between 5 and 10 µm → freq between 0.1 and 0.2 µm⁻¹)
        # min_freq = 1 / 10    # 0.1 µm⁻¹
        # max_freq = 1 / 5    # 0.2 µm⁻¹
        # # Mask to keep peaks in desired frequency range
        # valid_peaks = [p for p in peaks if min_freq <= pos_freqs[p] <= max_freq]
        # if valid_peaks:
        #     # Pick the strongest peak within range
        #     best_peak_idx = max(valid_peaks, key=lambda p: pos_power[p])
        #     peak_freq = pos_freqs[best_peak_idx]
        #     spacing_um_fft = 1 / peak_freq
        #     # Optional: Estimate error
        #     results = peak_widths(pos_power, [best_peak_idx], rel_height=0.5)
        #     width_freq = results[0][0]
        #     spacing_um_fft_err = spacing_um_fft * (width_freq / best_peak_idx)
        # else:
        #     spacing_um_fft = spacing_um_fft_err = None

        width_freq = peak_widths(pos_power, peaks, rel_height=0.5)[0][0]
        spacing_um_fft_err = spacing_um_fft * (width_freq / peaks[0])
    else:
        spacing_um_fft = spacing_um_fft_err = None

    # === AUTOCORRELATION ANALYSIS ===
    # The power spectrum is equivalent to squaring the magnitude of the Fourier transform.
    # We take the inverse 2D FFT (ifft2) of the power spectrum to get the autocorrelation function.
    # This gives a 2D autocorrelation image that shows how the original image correlates with itself at different spatial shifts in both X and Y directions.
    power_spec = np.abs(f)**2
    autocorr = fftshift(ifft2(power_spec).real) # .real removes imaginary part (artifacts)
    autocorr /= autocorr.max()  # Normalize so tha max value is 1 to simplify peak detection

    # Project along columns (i.e., vertical autocorrelation)
    # Since we are interested in vertical spacing (due to horizontal lines), we average the autocorrelation image across columns resulting in a 1D profile along Y (vertical axis).
    # This gives a signal representing how correlated each vertical shift is. (autocorrelation along Y axis)
    autocorr_y = autocorr.mean(axis=1)
    lags = np.arange(-h//2, h//2) * pixel_size_um  # This gives the actual shift in microns (lags) corresponding to each point in the autocorrelation curve. Ranges from -height/2 to +height/2 in steps of 1 pixel × pixel size.
    center_idx = len(lags) // 2
    lags_positive = lags[center_idx:] # Since the autocorrelation function is symmetric, we only analyze the positive side (i.e., positive shifts).
    autocorr_y_positive = autocorr_y[center_idx:]

    # Find autocorrelation peaks
    # peaks in the 1D autocorrelation curve that represent repeating patterns. The first prominent peak (after zero lag) corresponds to the first repeat distance = vertical spacing between horizontal lines.
    # The position of the first peak gives the estimated vertical spacing in microns. The FWHM (full width at half maximum) of the peak gives an error estimate, based on how sharp that peak is
    peaks_auto, _ = find_peaks(autocorr_y_positive, height=0.2, distance=20)
    if len(peaks_auto) > 0:
        spacing_um_auto = lags_positive[peaks_auto[0]]
        width_lag = peak_widths(autocorr_y_positive, peaks_auto, rel_height=0.5)[0][0]
        spacing_um_auto_err = spacing_um_auto * (width_lag / peaks_auto[0])
    else:
        spacing_um_auto = spacing_um_auto_err = None

    # === PLOTTING ===
    plt.figure(figsize=(14, 4))

    # FFT Spectrum
    plt.subplot(1, 3, 1)
    plt.imshow(np.log1p(np.abs(fshift)), cmap='gray')
    plt.title(f"{filename}\nFFT Spectrum (log scale)")

    # Power Spectrum Plot
    plt.subplot(1, 3, 2)
    plt.plot(pos_freqs, pos_power)
    if len(peaks) > 0:
        plt.plot(pos_freqs[peaks], pos_power[peaks], "rx")
    plt.title("Mean Power Spectrum (Y axis)")
    plt.xlabel("Spatial Frequency (1/µm)")
    plt.ylabel("Power")

    # Autocorrelation Plot
    plt.subplot(1, 3, 3)
    plt.plot(lags_positive, autocorr_y_positive)
    if len(peaks_auto) > 0:
        plt.plot(lags_positive[peaks_auto], autocorr_y_positive[peaks_auto], "rx")
    plt.title("Autocorrelation (Y axis)")
    plt.xlabel("Lag (µm)")
    plt.ylabel("Correlation")

    plt.tight_layout()
    plot_filename = os.path.splitext(filename)[0] + "_analysis.png"
    plt.savefig(os.path.join(output_folder, plot_filename), dpi=200)
    plt.close()

    # === SAVE RESULTS ===
    results.append({
        "Filename": filename,
        "FFT_Spacing_um": spacing_um_fft,
        "FFT_Spacing_Error_um": spacing_um_fft_err,
        "Autocorr_Spacing_um": spacing_um_auto,
        "Autocorr_Spacing_Error_um": spacing_um_auto_err
    })

# === WRITE RESULTS TO CSV ===
results_df = pd.DataFrame(results)
csv_path = os.path.join(output_folder, "spacing_results.csv")
results_df.to_csv(csv_path, index=False)

print(f"\nAll results saved to: {csv_path}")


Processing: MAX_ExM_ptetWT_SR_cell4_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell5_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell24_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell15_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell14_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell19_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell19_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell21_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell3_dorsal_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell18_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell25_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell27_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell6_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell22_ventral_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell2_dorsal2_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell22_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell2_dorsal_singlets.png
Processing: MAX_ExM_ptetWT

In [ ]:
# FOR ONLY ONE IMAGE, TO TEST
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, fftshift, ifft2, fft, fftfreq
from scipy.signal import find_peaks, peak_widths

# === SETTINGS ===
image_path = "W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_SR\\Maximum projections\\ROIs for analysis\\MAX_ExM_ptetWT_SR_cell1_dorsal_singlets_binaryridges-5.png"
pixel_size_um = 0.288  # microns per pixel

# === LOAD IMAGE ===
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Could not load image at: {image_path}")
h, w = img.shape

# === FFT ANALYSIS ===
f = fft2(img)
fshift = fftshift(f)
power_spectrum = np.abs(fshift)**2

# 1. FFT along vertical axis
fft_y = fft(img, axis=0)
power_y = np.abs(fft_y)**2

# 2. Mean across columns and shift
mean_power_y = fftshift(power_y.mean(axis=1))
freqs_y = fftshift(fftfreq(h, d=pixel_size_um))

# 3. Use only positive frequencies
pos_mask = freqs_y > 0
pos_freqs = freqs_y[pos_mask]
pos_power = mean_power_y[pos_mask]

# 4. Find peaks on the aligned data
peaks, _ = find_peaks(pos_power, distance=5, prominence=0.1)

if len(peaks) > 0:
    first_peak_freq = freqs_y[pos_mask][peaks[0]]
    spacing_um_fft = 1 / first_peak_freq

    results = peak_widths(mean_power_y[pos_mask], peaks, rel_height=0.5)
    width_freq = results[0][0]
    spacing_um_fft_err = spacing_um_fft * (width_freq / peaks[0])
else:
    spacing_um_fft = spacing_um_fft_err = None

# === AUTOCORRELATION ANALYSIS ===
power_spec = np.abs(f)**2
autocorr = fftshift(ifft2(power_spec).real)
autocorr /= autocorr.max()

# Analyze Y-direction autocorrelation
autocorr_y = autocorr.mean(axis=1)
lags = np.arange(-h//2, h//2) * pixel_size_um
center_idx = len(lags) // 2
lags_positive = lags[center_idx:]
autocorr_y_positive = autocorr_y[center_idx:]

peaks_auto, _ = find_peaks(autocorr_y_positive, height=0.1, distance=5)
if len(peaks_auto) > 0:
    spacing_um_auto = lags_positive[peaks_auto[0]]
    results_auto = peak_widths(autocorr_y_positive, peaks_auto, rel_height=0.5)
    width_lag = results_auto[0][0]
    spacing_um_auto_err = spacing_um_auto * (width_lag / peaks_auto[0])
else:
    spacing_um_auto = spacing_um_auto_err = None

# === PLOTTING ===
plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.imshow(np.log1p(np.abs(fshift)), cmap='gray')
plt.title("FFT Spectrum (log scale)")

plt.subplot(1, 3, 2)
plt.plot(pos_freqs, pos_power)
plt.plot(pos_freqs[peaks], pos_power[peaks], "rx")
plt.title("Mean Power Spectrum (Y direction)")
plt.xlabel("Spatial Frequency (1/µm)")
plt.ylabel("Power")

plt.subplot(1, 3, 3)
plt.plot(lags_positive, autocorr_y_positive)
plt.plot(lags_positive[peaks_auto], autocorr_y_positive[peaks_auto], "rx")
plt.title("Autocorrelation Profile (Y direction)")
plt.xlabel("Lag (µm)")
plt.ylabel("Correlation")

plt.tight_layout()
plt.show()

# === RESULTS ===
print("\n=== ESTIMATED SPACING ===")
if spacing_um_fft:
    print(f"FFT Estimate: {spacing_um_fft:.2f} µm ± {spacing_um_fft_err:.2f} µm (FWHM)")
else:
    print("FFT Estimate: No clear peak found.")

if spacing_um_auto:
    print(f"Autocorrelation Estimate: {spacing_um_auto:.2f} µm ± {spacing_um_auto_err:.2f} µm (FWHM)")
else:
    print("Autocorrelation Estimate: No clear peak found.")
